# 실험 02-02 다이오드 전파 정류회로

## 공통 코드

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

# %matplotlib tk
# %matplotlib inline

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def get_simulation_result(file_name, start=0, end=-1):
    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []
            
            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]
            
            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:7.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:7.3f}'.format(value))

def draw_plot(xs, ys, label, style_idx, marker_num=16):
    linestyle  = ['-', '-', '-', '-', '-', '-', '-', '-']
    colors     = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
    markers    = ['o', 'v', 's', 'p', '*', 'h', 'x', 'd']

    markeverys = []
    x_step = (xs.max() - xs.min()) / marker_num
    for idx in range(len(markers)):
        x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
        marker_list = []
        for j in range(marker_num):
            tmp_array = np.where(xs > (x_start + (x_step * j)))
            if (len(tmp_array[0]) > 0):
                marker_list.append(tmp_array[0][0])
        markeverys.append(marker_list)

    plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[style_idx], marker=markers[style_idx], markevery=markeverys[style_idx], label=label)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []

*******************************************************************************
## 시뮬레이션 1 결과 분석

## 회로도

* 사용한 컴포넌트
    + voltage (V): V1
    + res (R): R1
    + diode (D): D1
    + ground (G): Ground
    + Net Name (F4): VA, VB, VOUT
* 시뮬레이션 설정
    + SPICE directive (S)
        - .tran 0 100m 0 1u
* 기타 명령
    + 회전 (Control + R)
    + Wire (F3)
    + Move (F7)
    + Drag (F8)
    + Delete (F5)
    + Duplicate (F6)

    ![Markdown-mark](../ltspice/simulation_01.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_01.txt')

### 2. 시뮬레이션 결과에서 시간, VA, VB, 입력 전압(VA와 VB로 계산), 출력 전압 전압을 가져온다.

In [ ]:
times = data['TIME']
vas = data['V(VA)']
vbs = data['V(VB)']
vins = vas - vbs
vouts = data['V(VOUT)']

### 3. 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vas, 'V(VA)', 1)
draw_plot(times, vbs, 'V(VB)', 2)
draw_plot(times, vouts, 'V(OUT)', 3)
plt.show()

### 4. VA, VB, 출력 전압의 첨두값, 다이오드의 cut-in 전압을 구한다.

In [ ]:
V_A_peak_mV = vas.max()
V_B_peak_mV = vbs.max()
V_OUT_peak_mV = vouts.max()

print_value("V_A_peak_mV", V_A_peak_mV)
print_value("V_B_peak_mV", V_B_peak_mV)
print_value("V_OUT_peak_mV", V_OUT_peak_mV)

In [ ]:
vd1s = vas - vouts
vd2s = vbs - vouts

plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vd1s, 'V(D1)', 4)
draw_plot(times, vd2s, 'V(D2)', 5)
plt.show()

In [ ]:
V_D_cut_in_mV = max(vd1s.max(), vd2s.max())

print_value("V_D_cut_in_mV", V_D_cut_in_mV)

### 5. 시간을 x축으로 하고, 입력 전압, VA와 VB의 전압, 출력 전압을 y축으로 하는 그래프를 각각 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vas, 'V(VA)', 1)
draw_plot(times, vbs, 'V(VB)', 2)
plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vouts, 'V(OUT)', 3)
plt.show()

### 6. 출력 전압의 첨두값, 역방향 첨두값, 리플값을 구한다.

In [ ]:
idx_start = np.argmax(times > 20)
idx_end = np.argmin(times < 50)

vmax = vouts[idx_start:idx_end].max()
vmin = vouts[idx_start:idx_end].min()
V_OUT_peak_mV = vmax
V_OUT_reverse_peak_mV = -vmin
V_OUT_ripple_mV = vmax - vmin

print_value("V_OUT_peak_mV", V_OUT_peak_mV)
print_value("V_OUT_reverse_peak_mV", V_OUT_reverse_peak_mV)
print_value("V_OUT_ripple_mV", V_OUT_ripple_mV)

*******************************************************************************
## 시뮬레이션 2 결과 분석

* 회로
    + voltage (v): V1
    + res (r): R1
    + diode (d): D1
    + cap (c): C1
    + ground (g): Ground
    + Net Name (F4): VA, VB, VOUT
* 시뮬레이션 설정
    + SPICE directive (S)
        - .tran 0 100m 0 1u
        - .step param Cvar list 10uF, 22uF, 33uF, 47uF, 56uF
* 기타 명령
    + 회전 (Control + R)
    + Wire (F3)
    + Move (F7)
    + Drag (F8)
    + Delete (F5)
    + Duplicate (F6)

![Markdown-mark](../ltspice/simulation_02_sweep_c.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_02_sweep_c.txt')

### 2. 시뮬레이션 결과에서 시간, VA, VB, 입력 전압(VA와 VB로 계산), 출력 전압 전압을 가져온다.

In [ ]:
times = []
vas = []
vbs = []
vins = []
vouts = []
labels = ['10uF', '22uF', '33uF', '47uF', '56uF']

for idx in range(1, len(labels)+1):
    times.append(data['(TIME)@%d' % idx])
    vas.append(data['(V(VA))@%d' % idx])
    vbs.append(data['(V(VB))@%d' % idx])
    vins.append(vas[idx-1] - vbs[idx-1])
    vouts.append(data['(V(VOUT))@%d' % idx])

### 3. 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(labels)):
    draw_plot(times[idx], vins[idx], 'V(IN)@%d' % idx, 0)
    draw_plot(times[idx], vas[idx], 'V(VA)%d' % idx, 1)
    draw_plot(times[idx], vbs[idx], 'V(VB)%d' % idx, 2)
    draw_plot(times[idx], vouts[idx], 'V(OUT)%d' % idx, 3)
plt.show()

In [ ]:
idx = 0
print("C = %s" % labels[idx])

plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times[idx], vins[idx], 'V(IN)', 0)
draw_plot(times[idx], vas[idx], 'V(VA)', 1)
draw_plot(times[idx], vbs[idx], 'V(VB)', 2)
draw_plot(times[idx], vouts[idx], 'V(OUT)', 3)
plt.show()

### 4. VA, VB, 출력 전압의 첨두값을 구한다.

In [ ]:
idx = 0
print("C = %s" % labels[idx])

V_A_peak_mV = vas[idx].max()
V_B_peak_mV = vbs[idx].max()
V_OUT_peak_mV = vouts[idx].max()

print_value("V_A_peak_mV", V_A_peak_mV)
print_value("V_B_peak_mV", V_B_peak_mV)
print_value("V_OUT_peak_mV", V_OUT_peak_mV)

### 5. 시간을 x축으로 하고, 입력 전압, VA의 전압, VB의 전압, 출력 전압을 y축으로 하는 그래프를 각각 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_IN_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(labels)):
    draw_plot(times[idx], vins[idx], labels[idx], idx)
plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_A_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(labels)):
    draw_plot(times[idx], vas[idx], labels[idx], idx)
plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_B_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(labels)):
    draw_plot(times[idx], vbs[idx], labels[idx], idx)
plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_OUT_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(labels)):
    draw_plot(times[idx], vouts[idx], labels[idx], idx)
plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_OUT_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(labels)):
    draw_plot(times[idx], vouts[idx], labels[idx], idx)

ax = plt.gca()
ax.set_xlim([25, 60])
ax.set_ylim([2000, 4000])

plt.show()

### 6. 출력 전압의 첨두값, 역방향 첨두값, 리플값을 구한다.

In [ ]:
V_OUT_peak_mV = []
V_OUT_r_peak_mV = []
V_OUT_ripple_mV = []
for idx in range(len(labels)):
    idx_start = np.argmax(times[idx] > 20)
    idx_end = np.argmin(times[idx] < 50)

    vmax = vouts[idx][idx_start:idx_end].max()
    vmin = vouts[idx][idx_start:idx_end].min()

    V_OUT_peak_mV.append(vmax)
    V_OUT_r_peak_mV.append(vmin)
    V_OUT_ripple_mV.append(vmax - vmin)

print("C = ", end='')
print(labels)
print_array("V_OUT_peak_mV", V_OUT_peak_mV)
print_array("V_OUT_r_peak_mV", V_OUT_r_peak_mV)
print_array("V_OUT_ripple_mV", V_OUT_ripple_mV)